In [47]:
import string
print 'walk'.capitalize() # Walk is good
print 'AWOL'.capitalize() # not good because Awol isn't a word
print '-ness'.capitalize()
print 'February'.capitalize()
al = string.lowercase[0:26]
print type(al)
# for x in al: print x
print 'Tuesday'[0] in al

Walk
Awol
-ness
February
<type 'str'>
False


In [49]:
# output mwl39195hw.txt  (having duplicates)
# and explore 39195 hw entries
# 2015-12-01

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs
import string

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fhw = codecs.open('E:\\1Now\\taglist\\mwl39195hwraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1}) #.limit(1000)


totalhw = 0 # total number of head words
for result in results:    
    word = result.get("hw")    
    if word:
        #print word
        fhw.write(word + '\n')
        totalhw += 1
    else:
        print "no head word"
        continue

fhw.close()
print "total head words: ", totalhw


# sort out by head words and output
# with homographs, 39195 head words have duplicates; unique number is 34398

# create mwl39195hw.txt
with codecs.open('E:\\1Now\\taglist\\mwl39195hwraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
lines = sorted(lines)
print 'hw unique number: ', len(set(lines))

fd = codecs.open('E:\\1Now\\taglist\\mwl39195hw.txt', 'w', 'utf-8')
cnt = 0
for line in lines:
    fd.write(line + '\n')
    cnt += 1
fd.close()
print cnt



# explore 39195 hws
with codecs.open('E:\\1Now\\taglist\\mwl39195hw.txt', 'r', 'utf-8') as f:
    words = f.read().splitlines()
    f.close()
print 'number of hws with homographs: ', len(words)
print 'number of unique hws: ', len(set(words))

# wordsC = [x.capitalize() for x in words]
wordsC = []
for word in words:
    # only capitalize words beginning with a lowercase letter. no need for 'Feburary'
    # Don't capitalize words like 'AWOL' -- Awol is not a word!
    if word[0] in string.lowercase[0:26]:
        wordsC.append(word.capitalize())
        cnt += 1
    
print 'number of words capitalized: ', len(wordsC)

# put words and wordsC together and get unique
# wordsTotal = sorted(set(words + wordsC))
# fd = codecs.open('E:\\1Now\\taglist\\mwl39195hwCap.txt', 'w', 'utf-8')
# for x in wordsTotal:
#     fd.write(x + '\n')
# fd.close()
# print len(wordsTotal)



total head words:  39195
hw unique number:  34398
39195
number of hws with homographs:  39195
number of unique hws:  34398
number of words capitalized:  37517
67125


In [6]:
# output mwl39195sum.txt  (having duplicates)
# hw entries and their summary (subhw, prn, idpvs, tags)
# 2015-11-27 2015-12-01

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\taglist\\mwl39195sumraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "subhw":1, "prn":1, "idpvs":1, "tag":1}) #.limit(1000)


totalhw = 0 # total number of head words
for result in results:    
    word = result.get("hw")    
    if word:
        #print word
        fdict.write(word)
        totalhw += 1
    else:
        print "no head word"
        continue

    # get items of result
    prns = result.get("prn")
    subhws = result.get("subhw")
    idpvs = result.get("idpvs")  
    tags = result.get('tag')
    
    if subhws:
        cnt = 1
        for subhw in subhws:
            if cnt == 1: fdict.write(' (' + subhw)                
            else: fdict.write(' ' + subhw)
            cnt += 1        
        fdict.write(')')
    
    if prns:
        cnt = 1
        for prn in prns:                 
            fdict.write(' /' + prn + '/')
            cnt += 1

    if idpvs:
        cnt = 1
        for idpv in idpvs:
            if cnt == 1: fdict.write('  [' + idpv)
            else: fdict.write(', ' + idpv)
            cnt += 1
        fdict.write(']')
            
    if tags:
        cnt = 1
        for tag in tags:
            if cnt == 1: fdict.write('  {' + tag)
            else: fdict.write(' ' + tag)
            cnt += 1
        fdict.write('}')

    fdict.write('\n')    

fhw.close()
fdict.close()
print "total head words: ", totalhw


# sort out and create mwl39195sum.txt
with codecs.open('E:\\1Now\\taglist\\mwl39195sumraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
lines = sorted(lines)
print 'hw summary unique number: ', len(set(lines))

fd = codecs.open('E:\\1Now\\taglist\\mwl39195sum.txt', 'w', 'utf-8')
cnt = 0
for line in lines:
    fd.write(line + '\n')
    cnt += 1
fd.close()
print cnt

total head words:  39195
hw unique number:  34398
39195
hw summary unique number:  38862
39195


In [14]:
# output mwlBenchmark.txt (mwBen) (no duplicates)

# format: derivatives of hw (inflections, or subhws);hw
# sort by 'derivatives;hw' and make every line unique
# 2015-11-28, 2015-11-30, 2015-12-01

# blazes;blaze
# blazing;blaze
# blazingly;blazing

# I can know blaze, blazing are hws, the two lines below are not allowd because hws have derivates:
# blaze;blaze
# blazing;blazing
# the two lines above are redundant.

# manually delete: asas

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\taglist\\mwlBenchmarkraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1}) #.limit(1000)
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)


hwcnt = 0 # number of head words (hws)
# hwcapcnt = 0 # number of hws capitalized
inflcnt = 0 # number of inflections
subhwcnt = 0 # number of subhws
subhw1cnt = 0 # number of subhws whose first letter is not same as hw
onlyhwcnt = 0

# subhw1hw = []

for result in results:
    inflflag = 0
    subhwflag = 0
    
    word = result.get("hw")    
    if word:
        #print word
#         fdict.write(word + ';' + word + '\n')
        hwcnt += 1      
    else:
        print "no head word"
        continue

    # get items of result
    infls = result.get("infl")
    subhws = result.get("subhw")     
    
    if infls:
        for infl in infls:
            if infl == word:
#                 print 'infl == word', infl
                continue
            fdict.write(infl + ';' + word + '\n')
            inflcnt += 1   
            inflflag = 1
            
    if subhws:
        for subhw in subhws:
            #if subhw[0] != word[0]: continue       
            if subhw == word:
#                 print 'subhw == word', infl
                continue           
            if subhw[0] != word[0]:
#                 print 'subhw is not like word: ', subhw, '...', word
                subhw1cnt += 1
#                 subhw1hw.append(subhw+';'+word)
            fdict.write(subhw + ';' + word + '\n')
            subhwcnt += 1      
            subhwflag = 1
    if inflflag == 0 and subhwflag == 0:
        fdict.write(word + ';' + word + '\n')
        onlyhwcnt += 1
    
fdict.close()

print '--- mwBen statistics ---'
print "total head words: ", hwcnt
# print 'total hws capitalized: ', hwcapcnt
print "total inflections: ", inflcnt
print 'total subhws: ', subhwcnt
print 'total subhws whose 1st letters are different from hws: ', subhw1cnt
print 'only hw without derivatives: ', onlyhwcnt


# sort out by head words
with codecs.open('E:\\1Now\\taglist\\mwlBenchmarkraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
lines = sorted(set(lines))

fd = codecs.open('E:\\1Now\\taglist\\mwlBenchmark.txt', 'w', 'utf-8')
cnt = 0
for line in lines:
    fd.write(line + '\n')
    cnt += 1
fd.close()
print 'number of lines in mwBen: ', cnt

# subhw1hw = sorted(set(subhw1hw))
# fd = codecs.open('E:\\1Now\\taglist\\subhw1hw.txt', 'w', 'utf-8')
# for x in subhw1hw:
#     fd.write(x + '\n')
# fd.close()


--- mwBen statistics ---
total head words:  39195
total inflections:  64355
total subhws:  9544
total subhws whose 1st letters are different from hws:  334
only hw without derivatives:  11182
number of lines in mwBen:  53972


In [8]:
import string
print string.uppercase[0:26]

ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [15]:
# explore mwBen

import codecs
import string

with codecs.open('E:\\1Now\\taglist\\mwlBenchmark.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()

deriv = [] # derivative
benhw = [] # hw
deriv1hw = [] # derivative (inflection, subhw) first letter different from its hw
hwcap = [] # hw whose first letter is capitalized
for line in lines:
    x1 = line.split(';')[0]
    x2 = line.split(';')[1]
    deriv.append(x1)
    benhw.append(x2)
    if x1[0] != x2[0]: deriv1hw.append(line)
    if x2[0] in string.uppercase[0:26]: hwcap.append(x2)
derivS = sorted(deriv)
benhwSU = sorted(set(benhw))
hwcap = sorted(set(hwcap))

print '--- explore mwBen ---'
print 'number of derivative column: ', len(deriv)
print 'number of hw column: ', len(benhw)
print 'unique number of hw column: ', len(benhwSU)
print 'number of derivatives (inflections and subhws) whose 1st letter different from hws: ', len(deriv1hw)
print 'number of capitalized hws: ', len(hwcap)

fout = codecs.open('E:\\1Now\\taglist\\mwlStat.txt', 'w', 'utf-8')
fout.write('--- statistics ---\n')
fout.write('number of lines in mwBen: ' + str(len(deriv)) + '\n')
fout.write('unique number of hw: ' + str(len(benhwSU)) + '\n')
fout.write('number of derivatives (inflections and subhws) whose 1st letter different from hws: ' + str(len(deriv1hw)) + '\n')
fout.write('number of capitalized hws: ' + str(len(hwcap)) + '\n')

fout.write('Contents\n--------\n')
fout.write("Q: Are there duplicate derivatives who aren't hws in mwBen?\n")
fout.write("Q: List derivatives (inflections and subhws) whose first letter is different from the first letter of their hws\n")
fout.write("Capitalized hws\n\n")

# Q: Are there duplicate derivatives who aren't hws in mwBen?
dupderiv = [x for x in derivS if x not in benhwSU and derivS.count(x)>1]
fout.write("--- Duplicate derivatives not being hws: ---\n")
for x in dupderiv:
    fout.write(x + '\n')

# Q: List subhws whose first letter is different from the first letter of their hws
fout.write("\n--- derivatives (inflections and subhws) whose 1st letter different from hws: ---\n")
fout.write("derivatives (inflections and subhws);hw\n\n")
for x in subhw1hw:
    fout.write(x + '\n')

fout.write("\n--- Capitalized hws: ---\n")
for x in hwcap:
    fout.write(x + '\n')
           
fout.close()    

--- explore mwBen ---
number of derivative column:  53972
number of hw column:  53972
unique number of hw column:  34398
number of derivatives (inflections and subhws) whose 1st letter different from hws:  337
number of capitalized hws:  1487


In [10]:
# get duplicates
l = [1,2,3,4,4,5,5,6,1]
set([x for x in l if l.count(x) > 1])

{1, 4, 5}

In [74]:
# output core31.txt by processing core3000raw20151123.txt

with codecs.open('E:\\1Now\\taglist\\core3000raw20151123.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
words = [x.split('\t')[0] for x in lines]
wordsu = sorted(set(words))
fout = codecs.open('E:\\1Now\\taglist\\core3140n.txt', 'w', 'utf-8')
for word in wordsu:
    fout.write(word + '\n')
fout.close()

print len(words), len(wordsu)
dupl = set([x for x in words if words.count(x)>1])
print len(dupl)
# print sorted(dupl)

# # compare core3140n.txt and core31.txt
# with codecs.open('E:\\1Now\\taglist\\core31.txt', 'r', 'utf-8') as f:
#     core31s = f.read().splitlines()
#     f.close()
# print set(core31s) ^ set(wordsu)
    

3905 3140
661
set([])


In [1]:
def gethw():
    a = [{'k1': 'hello', 'k2':'hello2'}, {'k1': 'hi', 'k2':'hi2'}]
    return a
b = gethw()
for x in b:
    print x['k1'], '...', x['k2']

hello ... hello2
hi ... hi2


In [30]:
# check word lists only for building mwl mongodb database
# these word lists include (number in parentheses are the number of files)
#   * words of 18 topics from mwl website (18)
#   * 3140 core words from mwl website (1)
#   * words of "The Spelling of Different Sounds in English" (1)
#   * words of word roots, prefixes, suffixes (3)
#   sub-total: 23
#
#   * words of mwl single images (1)
#   * words of mwl group images (106)
#   sub-total: 107
#
#   * homographs (1)
#   * 3787 core words I collect (1)
#   sub-total: 2
# 
#   Total: 131



import codecs
import re
import string


def checkBymwBen(path, wordlist):
    '''get sorted unique head words of mwl for a word list which may include inflections and/or sub hws'''
    
    hwreturn = [] # hws returned by this function    
    
    # word list I want to check
    with codecs.open(path + wordlist, 'r', 'utf-8') as f:
        txtraw = f.read().splitlines()
    print '------', wordlist, '------'
    print '* has words : ', len(txtraw)
    print '* has unique word number: ', len(set(txtraw))
    print '* has duplicates: ', set([x for x in txtraw if txtraw.count(x) > 1])
    print '\n'

    # with codecs.open("E:\\1Now\\taglist\\mwl77Entry.txt", 'r', 'utf-8') as f:
    with codecs.open("E:\\1Now\\taglist\\mwlBenchmark.txt", 'r', 'utf-8') as f:    
        lines = f.read().splitlines()
        f.close()

    # create mwBen dictionary
    mwBens = []
    hws = [] # sorted unique hws
    hwsC = [] # sorted unique capitalized hws
    for line in lines:
        entry = {}
        hw = line.split(';')[1]
        entry['hw'] = hw
        entry['deriv'] = line.split(';')[0] # derivative, or hw that has no derivatives
        mwBens.append(entry)
        hws.append(hw)
        if hw[0] in string.lowercase[0:26]:
            hwsC.append(hw.capitalize())
        
    hws = sorted(set(hws))
    hwsC = sorted(set(hwsC))        

#     print 'mwBens: ', len(mwBens)
#     print 'hws: ', len(hws)
#     print 'hwsC: ', len(hwsC)


    # begin to check the word list against mw benchmark dictionary
    #  Word; Its-hw (Head Word)
    #
    # Three cases (the first two cases return hw)
    # 1. word is exactly same as hw or is a capitalized hw
    # 2. word is a derivative or is a capitalized derivative
    # 3. word is not in mwBen


    wordhw = [] # word is exactly same as hw
    wordhwC = [] # word is same as capitalized hw
    wordderiv = [] # derivatives or capitalized derivatives
    wordnotBen = [] # word is not in mwBen

    for word in txtraw:
        hwflag = 0
        derivflag = 0
        
        if word in hws:
            wordhw.append(word)
            hwreturn.append(word)
            hwflag = 1
        elif word in hwsC:
            wordhwC.append(word)
            hwreturn.append(word.lower())
            hwflag = 1

        if hwflag == 0:
            dercnt = 0
            for mwben in mwBens:                
                if word == mwben['deriv'] or word == mwben['deriv'].capitalize():
                    wordderiv.append(word)
                    hwreturn.append(mwben['hw'])
                    derivflag = 1
                    print 'derivative: ', word, '--- hw: ', mwben['hw']
                    dercnt += 1
            if dercnt > 1: print word, '--- derivate has multiple hws --- ', dercnt

        if hwflag == 0 and derivflag == 0:
            wordnotBen.append(word)
    
    print '\n'
    print '* same as hw: ', len(wordhw)  #,' -- ', wordhw
    print '* same as capitalized hw: ', len(wordhwC)  #,' -- ', wordhwC
    print '* words are derivatives: ', len(wordderiv) #,' -- ', wordderiv
    print '* not in mwBen: ', len(wordnotBen), ' -- ', wordnotBen
    if len(txtraw) == len(wordhw) + len(wordhwC) + len(wordderiv) + len(wordnotBen): 
        print '--- len(txtraw) == len(wordhw) + len(wordhwC) + len(wordderiv) + len(wordnotBen) ---\n'
    else: print "--- words' numbers don't mattch, check ---\n"
    
    dupl = [x for x in hwreturn if hwreturn.count(x)>1]
    print 'how many duplicates in raw hwreturn --- ', len(dupl), ' - ', len(set(dupl)), ' = ', len(dupl)-len(set(dupl))
    print dupl

    print '\nhow many hws supposed to be returned--- ', len(txtraw) - len(wordnotBen) - (len(dupl)-len(set(dupl)))
    hwreturn = sorted(set(hwreturn))
    print 'acutal return: ', len(hwreturn)
    return hwreturn
    
    
result = checkBymwBen("E:\\1Now\\taglist\\originaltaglist\\", "core3140n.txt")
# result = checkBymwBen('E:\\1Now\\taglist\\', 'testmwBen.txt')

print len(result)
# print result

------ core3140n.txt ------
* has words :  3140
* has unique word number:  3140
* has duplicates:  set([])


derivative:  annoying --- hw:  annoy
derivative:  annually --- hw:  annual
derivative:  constantly --- hw:  constant
derivative:  contamination --- hw:  contaminate
derivative:  continually --- hw:  continual
derivative:  continuously --- hw:  continuous
derivative:  customs --- hw:  custom
derivative:  damages --- hw:  damage
derivative:  dietician --- hw:  dietitian
derivative:  doubles --- hw:  double
derivative:  extroverted --- hw:  extrovert
derivative:  flurries --- hw:  flurry
derivative:  frequently --- hw:  frequent
derivative:  impatience --- hw:  impatient
derivative:  infrequently --- hw:  infrequent
derivative:  introverted --- hw:  introvert
derivative:  loneliness --- hw:  lonely
derivative:  mediation --- hw:  mediate
derivative:  normally --- hw:  normal
derivative:  notes --- hw:  note
derivative:  previously --- hw:  previous
derivative:  sadness --- hw:  sad

In [29]:

def compareWordlist(path, wl1, wl2):
    '''get common and different parts of two word lists'''
    with codecs.open(path + wl1, 'r', 'utf-8') as f:
        words1 = f.read().splitlines()
    with codecs.open(path + wl2, 'r', 'utf-8') as f:
        words2 = f.read().splitlines()
    f = codecs.open('E:\\1Now\\taglist\\compare\\' + wl1 + '_' + wl2 + '.txt', 'w', 'utf-8')
    comm = [x for x in words1 if x in words2]
    only1 = [x for x in words1 if x not in words2]
    only2 = [x for x in words2 if x not in words1]
    
    f.write('words only in ' + wl1 + ': ' + str(len(only1)) + '\n')
    for x in only1:
        f.write(x + '\n')    
    
    f.write('\n\nwords only in ' + wl2 + ': ' + str(len(only2)) + '\n')
    for x in only2:
        f.write(x + '\n')       
    
    f.write('\n\ncommon words of ' + wl1 + ' and ' + wl2 + ': ' + str(len(comm)) + '\n')
    for x in comm:
        f.write(x + '\n')

compareWordlist("E:\\1Now\\taglist\\originaltaglist\\", "core3140n.txt", "core37.txt")

In [25]:

# is it necessary to close file in with statement?
with codecs.open('E:\\temp\\with.txt', 'w', 'utf-8') as f:
    f.write('hi')
    f.close() # no need, it'll be closed automatically
f.write('hello') # f has been closed automatically
f.close()

ValueError: I/O operation on closed file